In [13]:
# read the network details
def readNet(fileName):
    f = open(fileName, "r")
    net = {}
    n = int(f.readline())
    net['noNodes'] = n
    mat = []
    for i in range(n):
        mat.append([])
        line = f.readline()
        elems = line.split(",")
        for j in range(n):
            mat[-1].append(int(elems[j]))
    net["mat"] = mat
    f.close()
    return net

In [14]:
import tsplib95 as tsp
import networkx as nx

def read_tsp_file(file_name_input):
    tsp_problem = tsp.load_problem(file_name_input)
    G = tsp_problem.get_graph()
    n=len(G.nodes())
    net = {}
    net['noNodes']=n
    matrix=nx.to_numpy_matrix(G)
    net['mat']=matrix
    return net

#matrix[i][j] ---> matrix.item((i, j))

In [15]:
# define the function

def fcEval(representation):
    fitness=0
    noNodes = net['noNodes']
    matrix = net['mat']
    for i in range(noNodes-1):
        fitness+=matrix[representation[i]][representation[i+1]]
    fitness+=matrix[representation[-1]][representation[0]]
    return fitness

In [16]:
# define the function
#matrix[i][j] ---> matrix.item((i, j))

def fcEval2(representation):
    fitness=0
    noNodes = net['noNodes']
    matrix = net['mat']
    for i in range(noNodes-1):
        fitness+=matrix.item(representation[i],representation[i+1])
    fitness+=matrix.item(representation[-1],representation[0])
    return fitness


In [17]:
from Chromosome import Chromosome
from random import randint

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres)
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness < self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        

In [18]:
import numpy as np
#net=readNet("data/easy.txt")
net=read_tsp_file("data/eil.txt")

noDim=net["noNodes"]
MIN=0
MAX=noDim

gaParam = {'popSize' : 100, 'noGen' : 1000}

problParam = {'min' : MIN, 'max' : MAX, 'function' : fcEval2, 'noDim' : noDim}

ga = GA(gaParam, problParam)
ga.initialisation()
ga.evaluation()
bestFitness=noDim*9999
bestRepres=[]
fileName_output="eil_out.txt"
f=open(fileName_output,'w')
for g in range(gaParam['noGen']):
    bestSolX = ga.bestChromosome().repres
    bestSolY = ga.bestChromosome().fitness
    if bestSolY<bestFitness:
        bestFitness=bestSolY
        bestRepres=bestSolX
     
    ga.oneGeneration()
    ga.evaluation()
    f.write('Best solution in generation ' + str(g) + ' is: x = ' + str(bestSolX) + ' f(x) = ' + str(bestSolY))
    f.write('\n')
    
f.write("Best repres&fitness: "+str(bestRepres)+" "+str(bestFitness))
f.write('\n')
    
f.close()